In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

import pandas as pd
import numpy as np
import sys
import typing

sys.path.extend(['../src', '../config'])

region = boto3.Session().region_name
session = sagemaker.Session()
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

import helpers.instance as ins
import helpers.s3 as s3_helper
import helpers.utils as ut
import helpers.athena as at

# READ YAML FILES AND STORE RELEVANT PATHS in dictionaries
dataset_cfg = ins.read_config('../config/datasets.yaml') 
config_cfg = ins.read_config('../config/config.yaml')
sql_cfg = ins.read_config('../config/sql.yaml')

In [2]:
input_files = [dataset_cfg['raw']['repair_base']['input'], dataset_cfg['raw']['base_query']['input']]
output_paths = [dataset_cfg['raw']['repair_base']['output_paths'], dataset_cfg['raw']['base_query']['output_paths']]

In [10]:
output_paths

['s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Interim/repair_base/',
 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Interim/base_query/']

In [11]:
input_files

['../sql/repair_base.txt', '../sql/base_query.txt']

In [3]:
# passing Athena variables
database=['ptledw_playarea', 'ptl_maintenance', 'ptl_connfleet', 'proactive_maint_db', 'ptl_adhoc']
workgroup='WG-Advanalytics'
tablename=['d_location_master','constellation_rules','ufch_depup', 'archive_pd_inference_scenario_output', 'corp_foilanl']

In [5]:
#Query string from a file, essentially, can get rid of
#s3 = boto3.client('s3', region)
#bucket_name = dataset_cfg['bucket']
#file = dataset_cfg['raw']['base_query']['input_file']
#file_obj = s3.get_object(Bucket=bucket_name,Key=file) #retrieves from S3
#file_data = file_obj['Body'].read() #reads contents of file
#query_string = file_data.decode() #method used to decode strings, which the file is
'''
def read_query_file(file_paths) -> list:
    """
        Reads queries from file and returns a list of query strings
        Args:
            file_paths (str):
        Returns:
            query_strings (list|str):
    """
    query_strings = []
    file_paths = ['../sql/repair_base.txt', '../sql/base_query.txt']
    for input_file in file_paths:
        query_string = open('../sql/base_query.txt').read()
        query_string = query_string.replace('\n',' ')
        query_string = query_string.replace('\t',' ')
        query_strings.append(query_string)
    return query_strings
'''

'\ndef read_query_file(file_paths) -> list:\n    """\n        Reads queries from file and returns a list of query strings\n        Args:\n            file_paths (str):\n        Returns:\n            query_strings (list|str):\n    """\n    query_strings = []\n    file_paths = [\'../sql/repair_base.txt\', \'../sql/base_query.txt\']\n    for input_file in file_paths:\n        query_string = open(\'../sql/base_query.txt\').read()\n        query_string = query_string.replace(\'\n\',\' \')\n        query_string = query_string.replace(\'\t\',\' \')\n        query_strings.append(query_string)\n    return query_strings\n'

In [12]:
#utilize loop to conduct all four queries
#use function to create query strings
#query

query_strings = ut.read_query_file(input_files)
#query_string = open('../sql/base_query.txt').read()
#query_string = query_string.replace('\n',' ')
#query_string = query_string.replace('\t',' ')
#query_string
#query_string = 'select * from ptl_maintenance.constellation_rules'

In [7]:
query_strings[0]

"SELECT  UNIT.UNIT_NUM,  job.PARTITION_KEY PARTITION_KEY,     SUM(job.LABOR_HOURS) LABOR_HOURS,     SUM(job.PARTS_COST) PARTS_COST,     SUM(job.LABOR_COST) LABOR_COST,     SUM(job.OUTSIDE_COST) OUTSIDE_COST,     SUM(CASE       WHEN prof.PROFILE_ID in ('9012','9013','9014','T','G','D1','9359','689','M1','5582','5886',                                 '686','6436','687','6136','690','STATEINS','1084',                                 '9465',    '552',    '5701',    '5700',    '6019',    '6345',    '5724',                                 '9317',    '7830',    '6344') then 1             ELSE 0         END) AS Major_PM,     SUM((CASE       WHEN prof.PROFILE_ID in ('ELEC INJ','5889','9035','6756','8924','A/C COMP','RADASSY','RPLTANK2', 'TURBO' ) or             ( comp.COMP_CODE in ('041','042','043','044','045','026','027') and (PARTS_COST+LABOR_COST+OUTSIDE_COST) >=800) then 1       ELSE 0      END))AS big_repair,  SUM(CASE    WHEN comp.COMP_CODE IN ('061','057') THEN 1   ELSE 0  END ) AS acci

In [8]:
query_strings

["SELECT  UNIT.UNIT_NUM,  job.PARTITION_KEY PARTITION_KEY,     SUM(job.LABOR_HOURS) LABOR_HOURS,     SUM(job.PARTS_COST) PARTS_COST,     SUM(job.LABOR_COST) LABOR_COST,     SUM(job.OUTSIDE_COST) OUTSIDE_COST,     SUM(CASE       WHEN prof.PROFILE_ID in ('9012','9013','9014','T','G','D1','9359','689','M1','5582','5886',                                 '686','6436','687','6136','690','STATEINS','1084',                                 '9465',    '552',    '5701',    '5700',    '6019',    '6345',    '5724',                                 '9317',    '7830',    '6344') then 1             ELSE 0         END) AS Major_PM,     SUM((CASE       WHEN prof.PROFILE_ID in ('ELEC INJ','5889','9035','6756','8924','A/C COMP','RADASSY','RPLTANK2', 'TURBO' ) or             ( comp.COMP_CODE in ('041','042','043','044','045','026','027') and (PARTS_COST+LABOR_COST+OUTSIDE_COST) >=800) then 1       ELSE 0      END))AS big_repair,  SUM(CASE    WHEN comp.COMP_CODE IN ('061','057') THEN 1   ELSE 0  END ) AS acc

## Read from Athena

In [9]:
# read data from Athena, add to loop (get_datasets_from_athena)
#df = at.get_data_from_athena(region=region,  query_string=query_string, 
                                   #   database=database[0],
                                    #  workgroup=workgroup)
dfs = at.get_datasets_from_athena(region = region, query_strings = query_strings, 
                                 database = database[0], catalog = 'AwsDataCatalog', 
                                 workgroup = workgroup)

Checking query status
Current status: QUEUED
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: SUCCEEDED


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (0,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Checking query status
Current status: QUEUED
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: RUNNING
Current status: SUCCEEDED


## Persist as a parquet

In [10]:
dfs

[         UNIT_NUM  PARTITION_KEY  VEHICLE_AGE_TOTAL_MONTHS  \
 0          100001         201801                        44   
 1          100001         201802                        45   
 2          100001         201803                        46   
 3          100001         201804                        47   
 4          100001         201805                        48   
 ...           ...            ...                       ...   
 4839169  ZSS63809         202101                        49   
 4839170  ZSS63809         202102                        50   
 4839171  ZSS63809         202103                        51   
 4839172  ZSS63809         202104                        52   
 4839173  ZSS63809         202105                        53   
 
                       REGION                           AREA  \
 0        0832-WESTERN REGION                  0642-MOUNTAIN   
 1        0832-WESTERN REGION                  0642-MOUNTAIN   
 2        0832-WESTERN REGION                  064

In [11]:
#path = ['s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Interim/base_query/']
#wrapper function, list of paths and dataframes, put df in respective path
#path = dataset_cfg['raw']['base_query']['output_paths']
s3_helper.persist_files_to_path(dfs=dfs,
                       paths = output_paths,
                       filetype='parquet')

In [12]:
pd.read_parquet(output_paths[0])

,UNIT_NUM,PARTITION_KEY,VEHICLE_AGE_TOTAL_MONTHS,REGION,AREA,DISTRICT,LOCATION,UNIT_MAKE_CODE,UNIT_MODEL,UNIT_MODEL_YEAR,UNIT_CATEGORY,UNIT_SIZE,UNIT_BODY_TYPE,UNIT_FUEL_TYPE,ACT_VEH_INSERVDT,PRODUCT_LINE,UNIT_SOLD_DATE,flag_datatype,MILES
0,100001,201801,44,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,46108
1,100001,201802,45,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,46108
2,100001,201803,46,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,48116
3,100001,201804,47,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,48116
4,100001,201805,48,0832-WESTERN REGION,0642-MOUNTAIN,0756-10-SOUTH LAS VEGAS,0756-10-SOUTH LAS VEGAS,GMC,G33705,2014,TRUCK,LIGHT,2000 06,GAS,2014-06-03 00:00:00.000,LEASE,2019-09-16 00:00:00.000,model_build,50347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4839169,ZSS63809,202101,49,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17472
4839170,ZSS63809,202102,50,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17491
4839171,ZSS63809,202103,51,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17491
4839172,ZSS63809,202104,52,0006-CORPORATE,0999-ADMINISTRATION AGGREGATE,6990-10-PENSKE FLEET MANAGEMENT,6990-10-PENSKE FLEET MANAGEMENT,HIN,268,2017,TRUCK,MEDIUM,1900 24,DIESEL,2017-01-01 00:00:00.000,CONTRACT MAINTENANCE - PEG,2021-02-17 00:00:00.000,model_build,17491


## Read from Parquet

In [13]:
df2 = s3_helper.read_multiple_parquet_files(output_paths)
df2

[         UNIT_NUM PARTITION_KEY VEHICLE_AGE_TOTAL_MONTHS               REGION  \
 0          100001        201801                       44  0832-WESTERN REGION   
 1          100001        201802                       45  0832-WESTERN REGION   
 2          100001        201803                       46  0832-WESTERN REGION   
 3          100001        201804                       47  0832-WESTERN REGION   
 4          100001        201805                       48  0832-WESTERN REGION   
 ...           ...           ...                      ...                  ...   
 4839169  ZSS63809        202101                       49       0006-CORPORATE   
 4839170  ZSS63809        202102                       50       0006-CORPORATE   
 4839171  ZSS63809        202103                       51       0006-CORPORATE   
 4839172  ZSS63809        202104                       52       0006-CORPORATE   
 4839173  ZSS63809        202105                       53       0006-CORPORATE   
 
              